In [ ]:
!pip install -U "tensorflow-text==2.13.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 30.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oa

In [ ]:
# !pip install transformers
import tensorflow as tf

In [ ]:
from transformers import BertModel, BertTokenizer
import tensorflow_hub as hub

In [ ]:
preprocess_url = "https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3"
encoder_url = "https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-10-h-128-a-2/versions/2"

In [ ]:
!pip install -q -U "tensorflow-text==2.13.*"


In [ ]:
import tensorflow_text as text

In [ ]:
bert_preprocess = hub.KerasLayer(preprocess_url)

In [ ]:
import pandas as pd
df = pd.read_csv('/content/sample_data/cleaned_file.csv')
df.head(1)

,text,label,category
0,FLASH SALE | LIMITED TIME ONLY Shop Now,1,Urgency


In [ ]:
import numpy as np

df.label.nunique(dropna = True)
df['label'] = df['label'].replace('Depends', 0.5)
temp = df['label']==0.5
df_filtered=df[~temp]
x = df_filtered['text']
labels=df_filtered['label'].astype(float)

y= np.asarray(labels, dtype=np.float32)

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [ ]:
original_text = df_filtered['text'].astype(str).tolist()


In [ ]:
preprocessed_text = bert_preprocess(original_text)

In [ ]:
bert_model = hub.KerasLayer(encoder_url)

In [ ]:
bert_model.trainable = True


In [ ]:

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
bert_results = bert_model(preprocessed_text)
pooled_output = bert_results['pooled_output']
# neural network layers
dense_layer_1 = tf.keras.layers.Dense(600, activation='relu', name='dense_layer_1')(pooled_output)

dense_layer_2 = tf.keras.layers.Dense(400, activation='relu', name='dense_layer_2')(dense_layer_1)
dense_layer_3 = tf.keras.layers.Dense(200, activation='relu', name='dense_layer_3')(dense_layer_2)
dense_layer_4 = tf.keras.layers.Dense(100, activation='relu', name='dense_layer_4')(dense_layer_3)
dropout_layer_1 = tf.keras.layers.Dropout(0.2, name='dropout_1')(dense_layer_4)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(dropout_layer_1)

bert_model.trainable = True

model = tf.keras.Model(inputs=[text_input], outputs = [output_layer])

In [ ]:

from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=1e-4)
model.compile(optimizer=opt,
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
85/85 [==============================] - 224s 2s/step - loss: 0.6054 - accuracy: 0.6549
Epoch 2/5
85/85 [==============================] - 211s 2s/step - loss: 0.5329 - accuracy: 0.6868
Epoch 3/5
85/85 [==============================] - 188s 2s/step - loss: 0.5022 - accuracy: 0.7027
Epoch 4/5
85/85 [==============================] - 189s 2s/step - loss: 0.4940 - accuracy: 0.6934
Epoch 5/5
85/85 [==============================] - 200s 2s/step - loss: 0.4800 - accuracy: 0.7190


In [ ]:
model.evaluate(x_test,y_test)

37/37 [==============================] - 32s 850ms/step - loss: 0.5526 - accuracy: 0.6347


[0.5525611639022827, 0.6347150206565857]

In [ ]:
# model.fit(x_train,y_train,epochs=5)

Epoch 1/5
109/109 [==============================] - 151s 1s/step - loss: 0.4615 - accuracy: 0.7007
Epoch 2/5
109/109 [==============================] - 135s 1s/step - loss: 0.4229 - accuracy: 0.7308
Epoch 3/5
109/109 [==============================] - 135s 1s/step - loss: 0.4237 - accuracy: 0.7308
Epoch 4/5
109/109 [==============================] - 135s 1s/step - loss: 0.4236 - accuracy: 0.7156
Epoch 5/5
109/109 [==============================] - 135s 1s/step - loss: 0.4160 - accuracy: 0.7285


In [ ]:
# model.evaluate(x_test,y_test)


13/13 [==============================] - 9s 480ms/step - loss: 0.5161 - accuracy: 0.6710


[0.5161225199699402, 0.6709844470024109]

In [ ]:
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict
k_folds = KFold(n_splits = 5)
from keras.wrappers.scikit_learn import KerasClassifier

ModuleNotFoundError: No module named 'keras.wrappers'

In [ ]:
wrapped_model = KerasWrapper(model)

scores = cross_val_score(wrapped_model, X, y, cv=5)
# score = cross_val_score((model),x,y,cv=k_folds,scoring="accuracy")

TypeError: Cannot clone object '<keras.src.engine.functional.Functional object at 0x7dba5d60d8a0>' (type <class 'keras.src.engine.functional.Functional'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [ ]:
model.predict(x_test)

13/13 [==============================] - 9s 607ms/step


array([[0.47044376],
       [0.00375732],
       [0.00311652],
       [0.29535383],
       [0.507007  ],
       [0.00315284],
       [0.5485225 ],
       [0.4785877 ],
       [0.519841  ],
       [0.25030228],
       [0.6459043 ],
       [0.48318237],
       [0.52493596],
       [0.00382598],
       [0.00334532],
       [0.01851062],
       [0.52484286],
       [0.26805884],
       [0.0058852 ],
       [0.4863988 ],
       [0.48234403],
       [0.02615674],
       [0.01859503],
       [0.48100075],
       [0.4994329 ],
       [0.4724032 ],
       [0.53228515],
       [0.00319911],
       [0.4956001 ],
       [0.00363244],
       [0.49179903],
       [0.0035159 ],
       [0.49318486],
       [0.49706474],
       [0.05852051],
       [0.86437726],
       [0.2287902 ],
       [0.16818558],
       [0.8050689 ],
       [0.5104344 ],
       [0.47265288],
       [0.49007708],
       [0.13549842],
       [0.53391176],
       [0.00364791],
       [0.00491311],
       [0.1819188 ],
       [0.576

In [ ]:
y_test

array([1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0.,
       1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 0.

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver  # Optional, if the content is loaded dynamically

In [ ]:
def get_page_content(url):
    # Use requests to get the HTML content of the page
    response = requests.get(url)
    return response.content

def get_dynamic_page_content(url):
    # Use Selenium if the content is loaded dynamically
    driver = webdriver.Chrome()  # Adjust this based on your browser
    driver.get(url)
    content = driver.page_source
    driver.quit()
    return content

def extract_keywords(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    text_content = soup.get_text()
    text_content = text_content.replace("\n","")
    #text_content = text_content.replace(" ","")
    print(type(text_content))
    for text in text_content.split('.'):
      print(text)
      predictions = model.predict([text])
      if predictions[0] >= 0.50000:
        print("Detected")
      else :
        print("not detected")


    # predictions = model.predict([text_content])

    # for i, prediction in enumerate(predictions):
    #   if prediction > 0.5:
    #     print("Detected")
    #   else:
    #     print(" Not Detected")

    # for keyword in keywords:
    #     if keyword.lower() in text_content.lower():
    #         print(f"Found keyword: {keyword}")



In [ ]:
ecommerce_url = "https://www.aachho.com/products/hyena-handblock-tiered-suit-set?utm_source=google&utm_medium=cpc&utm_campaign=WWWA_3519_adyogi_PerformanceMax_Top+SKUs-20275808049&gad_source=1&gclid=CjwKCAiA8NKtBhBtEiwAq5aX2FYO3fU2HI9kMQQAsZ2sVpZvVaXi1xAvTu_lmwaljKydTrJDuE3FthoCCfgQAvD_BwE"  # Replace with the actual URL of the eCommerce page

    # Get the HTML content of the page
html_content = get_page_content(ecommerce_url)

    # If the content is loaded dynamically, use the following instead
    # html_content = get_dynamic_page_content(ecommerce_url)

    # Extract and check for keywords
extract_keywords(html_content)

<class 'str'>
Buy Hyena Handblock Tiered Suit Set online in India at Best Price | Aachho                        Skip to contentClose menuNewSaleBestseller                    Ethnic Wear                                                Suit Sets                                                          Kurta Sets                                                          Co Ord Set                                                          Sharara Sets                                                          Anarkali Suit                                                          Lehenga Sets                                                          Sarees                                                Western Wear                                                Dresses                                                          Co Ord Set                            Loungewear                    Kidswear                                                Ethnic Wear                                       

In [ ]:
model.predict(["Only 7 left in stock.",
               "1 in Stock",
               "Low Stock Alert: Only 8 left in inventory",
               "Only 3 items left",
               "87% offers claimed. Hurry up!",
               "96 ITEM(S) LEFT IN STOCK!",
               "Availability: Only 4 Left!",
               "HOT PRODUCT Over 250 people are looking at this item.",
               "Only 33 left",
               "40 users are currently active on our website just now.",
               "Limited Time: $10 Off! Discount Reflected In Price!",
               "NO THANKS, I DONÃ¢â‚¬â„¢T MIND MISSING OUT",
               "Someone in Sasebo, Japan just bought",
               "Items in your cart are in high demand.",
               "Root Touch Up + Brow Filler",
               "Hurry up! 6 people other than you have this product in their cart",
               "Leslie in USA just Purchased Special Design Exclusive Handmade Silver Ring",
               "Add to Your Gift and We'll Ship it For Free!",
               "Before you check out, can we tempt you?",
               "No thanks, I dont want a discount",
               "Someone from Portland, United States (US) purchased MODALERT recently.",
               "You may cancel your JustFab VIP Membership at any time. There is absolutely no cancellation fee. If you would like to continue receiving selections, then just skip the",
               "DON'T MISS ON THESE AWESOME ITEMS / DON'T MISS ON OUR TABLE DECORATION SYSTEM",
               "Someone from Portland, United States (US) purchased MODALERT recently."])

1/1 [==============================] - 1s 692ms/step


array([[0.3355847 ],
       [0.47902757],
       [0.50496143],
       [0.34243217],
       [0.51691836],
       [0.5278854 ],
       [0.496583  ],
       [0.5141395 ],
       [0.29535383],
       [0.49706474],
       [0.50840974],
       [0.48129255],
       [0.483408  ],
       [0.405017  ],
       [0.00333156],
       [0.60177094],
       [0.47998905],
       [0.44535357],
       [0.0511926 ],
       [0.4873477 ],
       [0.4861077 ],
       [0.46881774],
       [0.47891703],
       [0.4861077 ]], dtype=float32)